In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain.memory import ConversationBufferMemory
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

llm = ChatOpenAI(temperature=0.1)

# 텍스트 파일 불러오는 방법
loader = UnstructuredFileLoader("./chapter_three.txt")

cache_dir = LocalFileStore(".cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
    memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up: \n\n {context}"),
    ("human", "context: {context}, question:{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

# RunnablePassthrough(): chain.invoke() 통해 넘어온 값을 입력해주는 역할
chain = (
    {
        "context": retriever, 
        "question": RunnablePassthrough()
    } | RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm 
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},  # 입력값
        {"output": result.content}  # 출력값
    )
    print(result)
    
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
content='He wrote "2+2=5" on the table.'
content='Julia is a character mentioned in the text who is associated with the protagonist and plays a significant role in his thoughts and actions.'


In [46]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
  HumanMessage(content='What message did he write in the table?'),
  AIMessage(content='He wrote "2+2=5" on the table.'),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content='Julia is a character mentioned in the text who is associated with the protagonist and plays a significant role in his thoughts and actions.')]}

(KR)
- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
- 체인을 수동으로 구현해야 합니다.
- 체인에 ConversationBufferMemory를 부여합니다.
- 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- 체인에 다음 질문을 합니다:
    - Aaronson 은 유죄인가요?
    - 그가 테이블에 어떤 메시지를 썼나요?
    - Julia 는 누구인가요?